In [57]:
import requests as re
import json
import pandas as pd
import time
import numpy as np
import math as mt

In [12]:
url = "http://comtrade.un.org"
numy = lambda x: f'0{x}' if x<10 else f'{x}'
products = ['%2C'.join([numy(i) for i in range(j*12+1,j*12+13)]) for j in range(2)]
countries = np.array([  4,   8,  12,  20,  24,  28,  31,  32,  36,  40,  44,  48,  50,
        51,  52,  56,  60,  68,  70,  72,  76,  84,  90,  96, 100, 104,
       108, 112, 116, 120, 124, 132, 136, 140, 144, 152, 156, 170, 174,
       178, 180, 188, 191, 196, 203, 204, 208, 214, 218, 222, 231, 233,
       242, 246, 251, 258, 268, 270, 275, 276, 288, 296, 300, 304, 308,
       320, 324, 328, 340, 344, 348, 352, 360, 364, 372, 376, 381, 384,
       388, 392, 398, 400, 404, 410, 414, 417, 418, 422, 426, 428, 440,
       442, 446, 450, 454, 458, 462, 470, 478, 480, 484, 490, 496, 498,
       499, 504, 508, 512, 516, 524, 528, 533, 540, 554, 558, 562, 566,
       579, 585, 586, 591, 600, 604, 608, 616, 620, 634, 642, 643, 646,
       659, 662, 670, 678, 682, 686, 688, 690, 694, 699, 702, 703, 704,
       705, 710, 716, 724, 729, 740, 748, 752, 757, 764, 768, 780, 784,
       788, 792, 800, 804, 807, 818, 826, 834, 842, 854, 858, 882, 887,
       894])
cpr = 5
cocountries = ['%2C'.join([f'{country}' for country in countries[j*cpr:cpr*(1+j)]]) for j in range(mt.ceil(len(countries)/cpr))]

In [163]:
total_requests = 0
start = time.time()
failed_combinations = []

In [ ]:
for year in range(1991,2001):
    all_data = [] (2525 last request length)
    for cocountry in cocountries:
        for product in products:
            result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={cocountry}&p=all&rg=1&cc={product}")
            total_requests += 1
            data = result.json()['dataset']
            print(len(data), 'len data')
            if len(data) == 0 or len(data) == 10000:
                fail = (year,cocountry,product)
                print(f'Adding failed request {fail}')
                failed_combinations.append(fail)
            all_data.extend(data)
            # Reset everything if more than an hour has elapsed
            if now - start > 3600:
                start = now
                total_requests = 0
            # If the number of requests
            if total_requests > 99: #and (now-start < 3600): Alredy accounted for in the if statement above
                time.sleep(3600-now + start + 10)
                start = time.time()
                total_requests = 0
    df = pd.DataFrame(all_data)
    df.to_csv(f'dummy_{year}.csv')
            

In [39]:
all_records = []
# TODO: sometimes the api answer is empty, but if we try with a smaller range of years, it is non-empty. FIX
for country in countries:
    if country <= 246: continue
    country_rec = []
    years_search = 10
    periods = mt.ceil(20/years_search)
    for year in ['%2C'.join(str(year) \
            for year in range(2001+k*years_search,2001+(k+1)*years_search)) for k in range(periods)]:#years:
        for product in products:
            # Get data
            result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={country}&p=all&rg=1&cc={product}")
            data = result.json()['dataset']
            time.sleep(1)

            # Validate obtained data is less than the api limit
            print(f'entries: {len(data)}')
            if len(data) == 10000 or len(data) == 0:
                print(f'Data may not be complete for code {country}')

            # Add new entries
            all_records.extend(data)
            country_rec.extend(data)
            
            # Account for the request just performed
            total_requests += 1
            now = time.time()
            
            # Reset everything if more than an hour has elapsed
            if now - start > 3600:
                start = now
                total_requests = 0

            # If the number of requests
            if total_requests > 99: #and (now-start < 3600): Alredy accounted for in the if statement above
                time.sleep(3600-now + start + 10)
                start = time.time()
                total_requests = 0

    # Save the country data
    df = pd.DataFrame(country_rec)
    print(f'Writing to memory country with code {country}')
    df.to_csv(f'{country}_2001-2020.csv')

2001%2C2002%2C2003%2C2004%2C2005
entries: 1887
entries: 2193
2006%2C2007%2C2008%2C2009%2C2010
entries: 1876
entries: 2404
2011%2C2012%2C2013%2C2014%2C2015
entries: 2125
entries: 2627
2016%2C2017%2C2018%2C2019%2C2020
entries: 2245
entries: 0
Data may not be complete for code 76
Writing to memory country with code 76


In [ ]:
result

In [ ]:
countries[186:]

In [20]:
total_requests

106

In [18]:
['%2C'.join(str(year) for year in range(2001+k*5,2006+k*5)) for k in range(4)]

['2001%2C2002%2C2003%2C2004%2C2005',
 '2006%2C2007%2C2008%2C2009%2C2010',
 '2011%2C2012%2C2013%2C2014%2C2015',
 '2016%2C2017%2C2018%2C2019%2C2020']

In [17]:
['%2C'.join([numy(i) for i in range(j*12+1,j*12+13)]) for j in range(2)]

['01%2C02%2C03%2C04%2C05%2C06%2C07%2C08%2C09%2C10%2C11%2C12',
 '13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C23%2C24']

In [53]:
year = '2016%2C2017%2C2018%2C2019%2C2020'
product = ['%2C'.join([numy(i) for i in range(j*12+1,j*12+13)]) for j in range(2)][1]
country = 76 #brazil
result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={country}&p=all&rg=1&cc={product}")
result

<Response [200]>

In [54]:
data = result.json()['dataset']

In [55]:
years = set([d['yr'] for d in data])
years

set()

In [56]:
total_requests = 0
start = time.time()

0

In [ ]:
year = 2015
all_records = []
# TODO: sometimes the api answer is empty, but if we try with a smaller range of years, it is non-empty. FIX
for country in countries:
    for product in products:
        # Get data
        result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={country}&p=all&rg=1&cc={product}")
        data = result.json()['dataset']
        time.sleep(1)
        # Validate obtained data is less than the api limit
        print(f'entries: {len(data)}')
        if len(data) == 10000 or len(data) == 0:
            print(f'Data may not be complete for code {country}')
        # Add new entries
        all_records.extend(data)
        country_rec.extend(data)            
        # Account for the request just performed
        total_requests += 1
        now = time.time()            
        # Reset everything if more than an hour has elapsed
        if now - start > 3600:
            start = now
            total_requests = 0
            # If the number of requests
        if total_requests > 99: #and (now-start < 3600): Alredy accounted for in the if statement above
            time.sleep(3600-now + start + 10)
            start = time.time()
            total_requests = 0

    # Save the country data
    df = pd.DataFrame(country_rec)
    print(f'Writing to memory country with code {country}')
    df.to_csv(f'{country}_2001-2020.csv')

In [78]:
time.sleep(3600)
year = 2015
ccc = 5
cocountries = ['%2C'.join([f'{country}' for country in countries[j*ccc:ccc*(1+j)]]) for j in range(mt.ceil(len(countries)/ccc))]
#all_data = [] (2525 last request length)
for cocountry in cocountries[14:]:
    for product in products:
        result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={cocountry}&p=all&rg=1&cc={product}")
        data = result.json()['dataset']
        print(len(data), 'len data')
        time.sleep(1)
        all_data.extend(data)

2524 len data
2636 len data
1152 len data
1353 len data
731 len data
1008 len data
2772 len data
2837 len data
1545 len data
1745 len data
2646 len data
2764 len data
2350 len data
2581 len data
1641 len data
1739 len data
1344 len data
1617 len data
1387 len data
1581 len data
1812 len data
2059 len data
1642 len data
1850 len data
2076 len data
2453 len data
1602 len data
1937 len data
830 len data
764 len data
914 len data
1321 len data
2143 len data
2364 len data
2032 len data
2248 len data
392 len data
586 len data
1714 len data
1896 len data
2758 len data
2893 len data
1227 len data
1296 len data
1966 len data
2150 len data
972 len data
1060 len data
2671 len data
2618 len data
1520 len data
1584 len data
1950 len data
2190 len data
422 len data
531 len data
1633 len data
1664 len data
1223 len data
1176 len data
389 len data
552 len data
919 len data
967 len data
2957 len data
3124 len data
338 len data
374 len data


In [136]:
df = pd.DataFrame(all_data)
df.to_csv(f'dummy_2015.csv')

In [72]:
cocountry

'266%2C268%2C270%2C276%2C288'

In [74]:
cocountries.index(cocountry)

14

In [77]:
cocountries[14:]

['266%2C268%2C270%2C276%2C288',
 '292%2C296%2C300%2C304%2C308',
 '312%2C320%2C324%2C328%2C332',
 '340%2C344%2C348%2C352%2C360',
 '364%2C368%2C372%2C376%2C380',
 '384%2C388%2C398%2C392%2C400',
 '404%2C408%2C410%2C414%2C417',
 '418%2C422%2C428%2C430%2C440',
 '446%2C450%2C454%2C458%2C466',
 '462%2C470%2C474%2C478%2C480',
 '484%2C490%2C496%2C498%2C500',
 '504%2C508%2C512%2C520%2C524',
 '527%2C528%2C530%2C533%2C540',
 '548%2C554%2C558%2C562%2C566',
 '570%2C574%2C579%2C583%2C584',
 '585%2C586%2C591%2C598%2C600',
 '604%2C608%2C616%2C624%2C620',
 '634%2C638%2C642%2C643%2C646',
 '654%2C659%2C660%2C662%2C666',
 '670%2C678%2C682%2C686%2C690',
 '694%2C699%2C702%2C704%2C705',
 '706%2C711%2C716%2C724%2C732',
 '736%2C740%2C752%2C757%2C760',
 '762%2C764%2C768%2C772%2C776',
 '780%2C784%2C788%2C792%2C795',
 '796%2C798%2C800%2C804%2C818',
 '826%2C834%2C854%2C860%2C858',
 '862%2C876%2C882%2C887%2C891',
 '899%2C894%2C203%2C231%2C434',
 '612%2C703%2C807%2C232%2C839',
 '72%2C175%2C275%2C334%2C336',
 '426%2C4

In [79]:
len(all_data)

162194

In [81]:
_countries = set([d['rtCode'] for d in all_data])
len(_countries)

168

In [134]:
table = df.groupby(['rtCode','ptCode','cmdCode']).size().reset_index().rename(columns={0:'Count'})

In [135]:
table.groupby(['Count']).size()

Count
1    161374
2      2524
dtype: int64

In [111]:
len(df.ptCode.unique())

244

In [112]:
len(countries)

239

In [113]:
ex_countries = np.array([country for country in df.ptCode.unique() if country not in countries])

In [114]:
ex_countries

array([  0, 381, 842, 581, 568, 652, 577,  16, 838, 316, 837,  74, 473,
        10, 535, 580, 637, 239])

In [120]:
time.sleep(3600)
for cocountry in [535, 580, 637, 239]:
    for product in products:
        result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={cocountry}&p=all&rg=1&cc={product}")
        data = result.json()['dataset']
        print(len(data), 'len data')
        time.sleep(1)
        all_data.extend(data)

0 len data
0 len data
0 len data
0 len data
0 len data
0 len data
0 len data
0 len data


In [119]:
cocountry

535

In [121]:
df = pd.DataFrame(all_data)

In [142]:
reporters = df.rtTitle.unique()
partners = df.ptTitle.unique()

In [145]:
g = [p for p in partners if p not in reporters]

In [149]:
len(reporters)

170

In [157]:
countries = df.rtCode.unique()
countries.sort()
countries

array([  4,   8,  12,  20,  24,  28,  31,  32,  36,  40,  44,  48,  50,
        51,  52,  56,  60,  68,  70,  72,  76,  84,  90,  96, 100, 104,
       108, 112, 116, 120, 124, 132, 136, 140, 144, 152, 156, 170, 174,
       178, 180, 188, 191, 196, 203, 204, 208, 214, 218, 222, 231, 233,
       242, 246, 251, 258, 268, 270, 275, 276, 288, 296, 300, 304, 308,
       320, 324, 328, 340, 344, 348, 352, 360, 364, 372, 376, 381, 384,
       388, 392, 398, 400, 404, 410, 414, 417, 418, 422, 426, 428, 440,
       442, 446, 450, 454, 458, 462, 470, 478, 480, 484, 490, 496, 498,
       499, 504, 508, 512, 516, 524, 528, 533, 540, 554, 558, 562, 566,
       579, 585, 586, 591, 600, 604, 608, 616, 620, 634, 642, 643, 646,
       659, 662, 670, 678, 682, 686, 688, 690, 694, 699, 702, 703, 704,
       705, 710, 716, 724, 729, 740, 748, 752, 757, 764, 768, 780, 784,
       788, 792, 800, 804, 807, 818, 826, 834, 842, 854, 858, 882, 887,
       894])

In [158]:
countries = np.array([  4,   8,  12,  20,  24,  28,  31,  32,  36,  40,  44,  48,  50,
        51,  52,  56,  60,  68,  70,  72,  76,  84,  90,  96, 100, 104,
       108, 112, 116, 120, 124, 132, 136, 140, 144, 152, 156, 170, 174,
       178, 180, 188, 191, 196, 203, 204, 208, 214, 218, 222, 231, 233,
       242, 246, 251, 258, 268, 270, 275, 276, 288, 296, 300, 304, 308,
       320, 324, 328, 340, 344, 348, 352, 360, 364, 372, 376, 381, 384,
       388, 392, 398, 400, 404, 410, 414, 417, 418, 422, 426, 428, 440,
       442, 446, 450, 454, 458, 462, 470, 478, 480, 484, 490, 496, 498,
       499, 504, 508, 512, 516, 524, 528, 533, 540, 554, 558, 562, 566,
       579, 585, 586, 591, 600, 604, 608, 616, 620, 634, 642, 643, 646,
       659, 662, 670, 678, 682, 686, 688, 690, 694, 699, 702, 703, 704,
       705, 710, 716, 724, 729, 740, 748, 752, 757, 764, 768, 780, 784,
       788, 792, 800, 804, 807, 818, 826, 834, 842, 854, 858, 882, 887,
       894])

,rtTitle,ptTitle
397,Andorra,Andorra
3242,Argentina,Argentina
3263,Australia,Australia
3347,Argentina,Argentina
3358,Australia,Australia
...,...,...
160284,South Africa,South Africa
160630,South Africa,South Africa
161010,South Africa,South Africa
161235,South Africa,South Africa
